# All process and test below!!.

In [1]:
# Import modules
from google.cloud import storage
import io
import regex as re
import pandas as pd
import numpy as np
import requests
import json
pd.set_option("display.max_columns", None)

In [2]:
# Define variables
storage_client = storage.Client.from_service_account_json("/Users/fiat/Desktop/Project/Clarissa_project/airflow/credentials/crs_credentials.json")
bucket_name = "clarissa-bucket"
blob_folder_path = "clarissa_raw_data"
sale_data = "clarissa_tiktok_sale_data.csv"
inbound_data = "inbound.csv"
revenue_data = "tiktok_revenue_data.csv"

In [3]:
# Use pandas to read csv from source
paths = [sale_data, revenue_data]
blob_paths = [f"{blob_folder_path}/{path}" for path in paths]
bucket = storage_client.bucket(bucket_name)

dataframes = []

for blob_path in blob_paths:
    blob = bucket.blob(blob_path)
    csv_file = blob.download_as_text()
    df = pd.read_csv(io.StringIO(csv_file), encoding='utf-8')
    dataframes.append(df)
    
sale_data_df = dataframes[0]
revenue_data_df = dataframes[1]


# Sale data transformation

Working on sale_data_df first (On this process 'Transform', Data engineer need to work with Owner of these datas)
- Find unused column and drop them
- Remove null values in Seller SKU column
- Transform anomaly value in Seller SKU column
- Remove 'THB' from currency columns
- Change date format
- Translate Thai Country, Province, district to English


In [4]:
# Check first overview of this dataframe
sale_data_df.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Normal or Pre-order,SKU ID,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Paid Time,RTS Time,Shipped Time,Delivered Time,Cancelled Time,Cancel By,Cancel Reason,Fulfillment Type,Warehouse Name,Tracking ID,Delivery Option,Shipping Provider Name,Buyer Message,Buyer Username,Recipient,Phone #,Zipcode,Country,Province,District,Detail Address,Additional address information,Payment Method,Weight(kg),Product Category,Package ID,Seller Note,Checked Status,Checked Marked by
0,577616838816336780,Shipped,In Transit,NaN,Normal,1729540466520722063,ABX08-D4,Hannah shirt - เสื้อคอปกผ้าร่องพรีเมียม,White,1,0,THB 190.00,THB 190.00,THB 0.00,THB 0.00,THB 190.00,THB 0.00,THB 35.00,THB 0.00,THB 35.00,THB 0.00,NaN,THB 190.00,NaN,22/07/2023 17:49:40\t,22/07/2023 17:50:09\t,22/07/2023 19:52:11\t,23/07/2023 15:26:32\t,\t,\t,NaN,NaN,Fulfillment by seller,Clarissa Warehouse,7.271803e+11,การจัดส่งมาตรฐาน,J&T Express,NaN,pavena607,ป***า ย***ี้ยง,(+66)981****03,63*20\t,ไทย,ตาก,บ้านตาก,93**************,NaN,Mbanking,0.2,Blouses & Shirts,1153353842960534412\t,NaN,Unchecked,NaN
1,577616009599683475,Shipped,In Transit,NaN,Normal,1729565289605008015,ACR01-A3,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,0,THB 260.00,THB 260.00,THB 0.00,THB 0.00,THB 260.00,THB 0.00,THB 35.00,THB 0.00,THB 35.00,THB 0.00,NaN,THB 260.00,NaN,22/07/2023 14:39:04\t,\t,22/07/2023 14:50:15\t,23/07/2023 15:26:32\t,\t,\t,NaN,NaN,Fulfillment by seller,Clarissa Warehouse,7.271720e+11,การจัดส่งมาตรฐาน,J&T Express,NaN,warangkanasmile,ส***ธากา ก***อบวิริยะ พ***รงเรียนวัดป่าตัน เ,(+66)061*****84,50*00\t,ไทย,เชียงใหม่,เมืองเชียงใหม่,1 ************************,โร***************,Cash on delivery,0.2,Jackets & Coats,1153353551189740435\t,NaN,Unchecked,NaN
2,577604062025123857,Canceled,Canceled,Cancel,Normal,1729565289605008015,ACR01-A3,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,1,THB 260.00,THB 260.00,THB 0.00,THB 120.00,THB 140.00,THB 0.00,THB 35.00,THB 0.00,THB 35.00,THB 0.00,NaN,THB 140.00,THB 140.00,20/07/2023 01:51:03\t,\t,\t,\t,\t,20/07/2023 02:55:11\t,User,Order created by mistake,Fulfillment by seller,Clarissa Warehouse,7.271079e+11,การจัดส่งมาตรฐาน,J&T Express,NaN,yp091520,ป***าวดี ศ***าพรม,(+66)937****92,83*00\t,ไทย,ภูเก็ต,เมืองภูเก็ต,52********************************************...,NaN,Cash on delivery,0.2,Jackets & Coats,1153349202350737425\t,NaN,Unchecked,NaN
3,577600144372763216,Shipped,Delivered,NaN,Normal,1729565292134369935,ACS02-D5,Cristine shirt - เสื้อเชิ้ตแขนยาวผูกคอ,Ocean blue,1,0,THB 190.00,THB 190.00,THB 0.00,THB 70.00,THB 120.00,THB 0.00,THB 35.00,THB 0.00,THB 35.00,THB 0.00,NaN,THB 120.00,NaN,19/07/2023 10:22:09\t,21/07/2023 16:38:34\t,19/07/2023 12:31:55\t,19/07/2023 13:59:32\t,21/07/2023 16:38:34\t,\t,NaN,NaN,Fulfillment by seller,Clarissa Warehouse,7.271745e+11,การจัดส่งมาตรฐาน,J&T Express,NaN,nasaneen28,น***ีน อ***,(+66)638****62,94*00\t,ราชอาณาจักรไทย,ปัตตานี,เมืองปัตตานี,36********************************************,NaN,Cash on delivery,0.2,Blouses & Shirts,1153347787199580752\t,NaN,Unchecked,NaN
4,577594882275051750,Shipped,Delivered,NaN,Normal,1729452639924030095,AAK05-A3,Harmony pants - กางเกงขายาวเดินตะเข็บ,"White, S",1,0,THB 290.00,THB 290.00,THB 0.00,THB 0.00,THB 290.00,THB 0.00,THB 35.00,THB 0.00,THB 35.00,THB 0.00,NaN,THB 290.00,NaN,18/07/2023 07:57:52\t,18/07/2023 07:58:22\t,18/07/2023 20:58:28\t,19/07/2023 13:59:32\t,21/07/2023 15:14:59\t,\t,NaN,NaN,Fulfillment by seller,Clarissa Warehouse,7.271715e+11,การจัดส่งมาตรฐาน,J&T Express,NaN,waris1110,น***าววริศรา ก***กวียน,(+66)061*****64,44*50\t,ไทย,มหาสารคาม,กันทรวิชัย,22********************************************...,NaN,

In [5]:
# Drop unused columns
sale_data_new_df = sale_data_df.drop(columns=[
    "Normal or Pre-order"
    , "SKU ID"
    , "Paid Time"
    , "RTS Time"
    , "Shipped Time"
    , "Delivered Time"
    , "Cancel Reason"
    , "Fulfillment Type"
    , "Warehouse Name"
    , "Delivery Option"
    , "Shipping Provider Name"
    , "Buyer Message"
    , "Recipient"
    , "Phone #"
    , "Zipcode"
    , "Detail Address"
    , "Additional address information"
    , "Weight(kg)"
    , "Package ID"
    , "Seller Note"
    , "Checked Status"
    , "Checked Marked by"
])

# Check dataframe after dropping columns and set Order ID as indexes
sale_data_new_df.head(5)

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Cancelled Time,Cancel By,Tracking ID,Buyer Username,Country,Province,District,Payment Method,Product Category
0,577616838816336780,Shipped,In Transit,NaN,ABX08-D4,Hannah shirt - เสื้อคอปกผ้าร่องพรีเมียม,White,1,0,THB 190.00,THB 190.00,THB 0.00,THB 0.00,THB 190.00,THB 0.00,THB 35.00,THB 0.00,THB 35.00,THB 0.00,NaN,THB 190.00,NaN,22/07/2023 17:49:40\t,\t,NaN,7.271803e+11,pavena607,ไทย,ตาก,บ้านตาก,Mbanking,Blouses & Shirts
1,577616009599683475,Shipped,In Transit,NaN,ACR01-A3,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,0,THB 260.00,THB 260.00,THB 0.00,THB 0.00,THB 260.00,THB 0.00,THB 35.00,THB 0.00,THB 35.00,THB 0.00,NaN,THB 260.00,NaN,22/07/2023 14:39:04\t,\t,NaN,7.271720e+11,warangkanasmile,ไทย,เชียงใหม่,เมืองเชียงใหม่,Cash on delivery,Jackets & Coats
2,577604062025123857,Canceled,Canceled,Cancel,ACR01-A3,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,1,THB 260.00,THB 260.00,THB 0.00,THB 120.00,THB 140.00,THB 0.00,THB 35.00,THB 0.00,THB 35.00,THB 0.00,NaN,THB 140.00,THB 140.00,20/07/2023 01:51:03\t,20/07/2023 02:55:11\t,User,7.271079e+11,yp091520,ไทย,ภูเก็ต,เมืองภูเก็ต,Cash on delivery,Jackets & Coats
3,577600144372763216,Shipped,Delivered,NaN,ACS02-D5,Cristine shirt - เสื้อเชิ้ตแขนยาวผูกคอ,Ocean blue,1,0,THB 190.00,THB 190.00,THB 0.00,THB 70.00,THB 120.00,THB 0.00,THB 35.00,THB 0.00,THB 35.00,THB 0.00,NaN,THB 120.00,NaN,19/07/2023 10:22:09\t,\t,NaN,7.271745e+11,nasaneen28,ราชอาณาจักรไทย,ปัตตานี,เมืองปัตตานี,Cash on delivery,Blouses & Shirts
4,577594882275051750,Shipped,Delivered,NaN,AAK05-A3,Harmony pants - กางเกงขายาวเดินตะเข็บ,"White, S",1,0,THB 290.00,THB 290.00,THB 0.00,THB 0.00,THB 290.00,THB 0.00,THB 35.00,THB 0.00,THB 35.00,THB 0.00,NaN,THB 290.00,NaN,18/07/2023 07:57:52\t,\t,NaN,7.271715e+11,waris1110,ไทย,มหาสารคาม,กันทรวิชัย,Truemoney,Trousers


In [6]:
# Find duplicated value in Order ID to check if I can use it as indexes or not.
# If there are duplicated value, I can't use it
duplicates_count = sale_data_new_df['Order ID'].duplicated().sum()
print(duplicates_count)

768


Remove THB from all columns that THB exist
I will use lambda function to apply str.replace

In [7]:
# List all column include 'THB'
cols_mod = [
    'SKU Unit Original Price',
    'SKU Subtotal Before Discount',
    'SKU Platform Discount',
    'SKU Seller Discount',
    'SKU Subtotal After Discount',
    'Shipping Fee After Discount',
    'Original Shipping Fee',
    'Shipping Fee Seller Discount',
    'Shipping Fee Platform Discount',
    'Taxes',
    'Small Order Fee',
    'Order Amount',
    'Order Refund Amount',
    ]

# Remove 'THB'
sale_data_new_df[cols_mod] = sale_data_new_df[cols_mod].apply(lambda x: x.str.replace('THB',''))

# Check output
sale_data_new_df.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Cancelled Time,Cancel By,Tracking ID,Buyer Username,Country,Province,District,Payment Method,Product Category
0,577616838816336780,Shipped,In Transit,NaN,ABX08-D4,Hannah shirt - เสื้อคอปกผ้าร่องพรีเมียม,White,1,0,190.00,190.00,0.00,0.00,190.00,0.00,35.00,0.00,35.00,0.00,NaN,190.00,NaN,22/07/2023 17:49:40\t,\t,NaN,7.271803e+11,pavena607,ไทย,ตาก,บ้านตาก,Mbanking,Blouses & Shirts
1,577616009599683475,Shipped,In Transit,NaN,ACR01-A3,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,0,260.00,260.00,0.00,0.00,260.00,0.00,35.00,0.00,35.00,0.00,NaN,260.00,NaN,22/07/2023 14:39:04\t,\t,NaN,7.271720e+11,warangkanasmile,ไทย,เชียงใหม่,เมืองเชียงใหม่,Cash on delivery,Jackets & Coats
2,577604062025123857,Canceled,Canceled,Cancel,ACR01-A3,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,1,260.00,260.00,0.00,120.00,140.00,0.00,35.00,0.00,35.00,0.00,NaN,140.00,140.00,20/07/2023 01:51:03\t,20/07/2023 02:55:11\t,User,7.271079e+11,yp091520,ไทย,ภูเก็ต,เมืองภูเก็ต,Cash on delivery,Jackets & Coats
3,577600144372763216,Shipped,Delivered,NaN,ACS02-D5,Cristine shirt - เสื้อเชิ้ตแขนยาวผูกคอ,Ocean blue,1,0,190.00,190.00,0.00,70.00,120.00,0.00,35.00,0.00,35.00,0.00,NaN,120.00,NaN,19/07/2023 10:22:09\t,\t,NaN,7.271745e+11,nasaneen28,ราชอาณาจักรไทย,ปัตตานี,เมืองปัตตานี,Cash on delivery,Blouses & Shirts
4,577594882275051750,Shipped,Delivered,NaN,AAK05-A3,Harmony pants - กางเกงขายาวเดินตะเข็บ,"White, S",1,0,290.00,290.00,0.00,0.00,290.00,0.00,35.00,0.00,35.00,0.00,NaN,290.00,NaN,18/07/2023 07:57:52\t,\t,NaN,7.271715e+11,waris1110,ไทย,มหาสารคาม,กันทรวิชัย,Truemoney,Trousers


In [8]:
# Remove all rows with no SKU ID
sale_data_new_df = sale_data_new_df.dropna(subset=['Seller SKU'])

# Check output
sale_data_new_df.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Cancelled Time,Cancel By,Tracking ID,Buyer Username,Country,Province,District,Payment Method,Product Category
0,577616838816336780,Shipped,In Transit,NaN,ABX08-D4,Hannah shirt - เสื้อคอปกผ้าร่องพรีเมียม,White,1,0,190.00,190.00,0.00,0.00,190.00,0.00,35.00,0.00,35.00,0.00,NaN,190.00,NaN,22/07/2023 17:49:40\t,\t,NaN,7.271803e+11,pavena607,ไทย,ตาก,บ้านตาก,Mbanking,Blouses & Shirts
1,577616009599683475,Shipped,In Transit,NaN,ACR01-A3,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,0,260.00,260.00,0.00,0.00,260.00,0.00,35.00,0.00,35.00,0.00,NaN,260.00,NaN,22/07/2023 14:39:04\t,\t,NaN,7.271720e+11,warangkanasmile,ไทย,เชียงใหม่,เมืองเชียงใหม่,Cash on delivery,Jackets & Coats
2,577604062025123857,Canceled,Canceled,Cancel,ACR01-A3,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,1,260.00,260.00,0.00,120.00,140.00,0.00,35.00,0.00,35.00,0.00,NaN,140.00,140.00,20/07/2023 01:51:03\t,20/07/2023 02:55:11\t,User,7.271079e+11,yp091520,ไทย,ภูเก็ต,เมืองภูเก็ต,Cash on delivery,Jackets & Coats
3,577600144372763216,Shipped,Delivered,NaN,ACS02-D5,Cristine shirt - เสื้อเชิ้ตแขนยาวผูกคอ,Ocean blue,1,0,190.00,190.00,0.00,70.00,120.00,0.00,35.00,0.00,35.00,0.00,NaN,120.00,NaN,19/07/2023 10:22:09\t,\t,NaN,7.271745e+11,nasaneen28,ราชอาณาจักรไทย,ปัตตานี,เมืองปัตตานี,Cash on delivery,Blouses & Shirts
4,577594882275051750,Shipped,Delivered,NaN,AAK05-A3,Harmony pants - กางเกงขายาวเดินตะเข็บ,"White, S",1,0,290.00,290.00,0.00,0.00,290.00,0.00,35.00,0.00,35.00,0.00,NaN,290.00,NaN,18/07/2023 07:57:52\t,\t,NaN,7.271715e+11,waris1110,ไทย,มหาสารคาม,กันทรวิชัย,Truemoney,Trousers


In [9]:
# Replace -.* with replace and check anomaly in column 'Seller SKU'
sale_data_new_df['Seller SKU'] = sale_data_new_df['Seller SKU'].str.replace('-.*', '', regex=True)

sale_data_new_df.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Cancelled Time,Cancel By,Tracking ID,Buyer Username,Country,Province,District,Payment Method,Product Category
0,577616838816336780,Shipped,In Transit,NaN,ABX08,Hannah shirt - เสื้อคอปกผ้าร่องพรีเมียม,White,1,0,190.00,190.00,0.00,0.00,190.00,0.00,35.00,0.00,35.00,0.00,NaN,190.00,NaN,22/07/2023 17:49:40\t,\t,NaN,7.271803e+11,pavena607,ไทย,ตาก,บ้านตาก,Mbanking,Blouses & Shirts
1,577616009599683475,Shipped,In Transit,NaN,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,0,260.00,260.00,0.00,0.00,260.00,0.00,35.00,0.00,35.00,0.00,NaN,260.00,NaN,22/07/2023 14:39:04\t,\t,NaN,7.271720e+11,warangkanasmile,ไทย,เชียงใหม่,เมืองเชียงใหม่,Cash on delivery,Jackets & Coats
2,577604062025123857,Canceled,Canceled,Cancel,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,1,260.00,260.00,0.00,120.00,140.00,0.00,35.00,0.00,35.00,0.00,NaN,140.00,140.00,20/07/2023 01:51:03\t,20/07/2023 02:55:11\t,User,7.271079e+11,yp091520,ไทย,ภูเก็ต,เมืองภูเก็ต,Cash on delivery,Jackets & Coats
3,577600144372763216,Shipped,Delivered,NaN,ACS02,Cristine shirt - เสื้อเชิ้ตแขนยาวผูกคอ,Ocean blue,1,0,190.00,190.00,0.00,70.00,120.00,0.00,35.00,0.00,35.00,0.00,NaN,120.00,NaN,19/07/2023 10:22:09\t,\t,NaN,7.271745e+11,nasaneen28,ราชอาณาจักรไทย,ปัตตานี,เมืองปัตตานี,Cash on delivery,Blouses & Shirts
4,577594882275051750,Shipped,Delivered,NaN,AAK05,Harmony pants - กางเกงขายาวเดินตะเข็บ,"White, S",1,0,290.00,290.00,0.00,0.00,290.00,0.00,35.00,0.00,35.00,0.00,NaN,290.00,NaN,18/07/2023 07:57:52\t,\t,NaN,7.271715e+11,waris1110,ไทย,มหาสารคาม,กันทรวิชัย,Truemoney,Trousers


SKU must be ^[a-zA-Z]{3}[0-9]{2}$ regex form (In this case I work with data dictionary)
so I will drop correct rows first and filter anomaly that I want to transform

In [17]:
pattern = r'^[a-zA-Z]{3}[0-9]{2}$'
sale_data_filtered = sale_data_new_df[~(sale_data_new_df['Seller SKU'].str.match(pattern, na=False))].drop_duplicates(subset=['Seller SKU'])
sale_data_filtered['Seller SKU'] = sale_data_filtered['Seller SKU'].str.replace(r'[\u0E00-\u0E7F]+', '', regex=True)
sale_data_filtered.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Cancelled Time,Cancel By,Tracking ID,Buyer Username,Country,Province,District,Payment Method,Product Category
1171,576723272411744995,Completed,Completed,NaN,ACF003,Pixie Shirt - เสื้อซิปแขนยาวกระเป๋าคู่ผ้าวัฟเฟิล,White,1,0,290.00,290.00,0.00,20.00,270.00,0.00,35.00,0.00,35.00,0.00,NaN,270.00,NaN,14/10/2022 22:33:16\t,\t,NaN,7.254733e+11,pae_zaaa,ราชอาณาจักรไทย,เทศบาลนครขอนแก่น,บ้านไผ่,Internet Banking,Blouses & Shirts
1180,576723233636386834,Completed,Completed,NaN,ACF001,Pixie Shirt - เสื้อซิปแขนยาวกระเป๋าคู่ผ้าวัฟเฟิล,Beige,1,0,290.00,290.00,0.00,20.00,270.00,0.00,35.00,0.00,35.00,0.00,NaN,270.00,NaN,14/10/2022 22:12:29\t,\t,NaN,7.254035e+11,tooktik_oo,ราชอาณาจักรไทย,เทศบาลเมืองยโสธร,คำเขื่อนแก้ว,Cash on delivery,Blouses & Shirts
1182,576723219452758089,Canceled,Canceled,Cancel,ACF002,Pixie Shirt - เสื้อซิปแขนยาวกระเป๋าคู่ผ้าวัฟเฟิล,Yellow,1,1,290.00,290.00,75.00,20.00,195.00,0.00,23.00,0.00,23.00,0.00,NaN,195.00,195.00,14/10/2022 22:05:07\t,15/10/2022 22:05:08\t,System,NaN,mayyazee,Thailand,Bangkok,Huai Khwang,NaN,Blouses & Shirts
2010,576631724884134303,Completed,Completed,NaN,ABN,Rainbow shirt - เสื้อเชิ้ตหลากสี(สลับสี),Mix,1,0,290.00,290.00,0.00,61.00,229.00,0.00,23.00,0.00,23.00,0.00,NaN,428.00,NaN,23/08/2022 23:01:25\t,\t,NaN,7.252636e+11,nat_thitinan,Thailand,Bangkok,Prawet,Credit Card/Debit Card,Blouses & Shirts
2199,576609354519513905,Completed,Completed,NaN,MA003,Mable Shirt-เสื้อเชิ้ตผ้าย่นคอตตอน,White,1,0,250.00,250.00,44.20,0.00,205.80,0.00,35.00,0.00,35.00,0.00,NaN,411.60,NaN,08/08/2022 23:27:43\t,\t,NaN,7.251561e+11,kiztyy,Thailand,Saraburi,Wang Muang,Truemoney,Blouses & Shirts


In [30]:
json_file_path = "/Users/fiat/Desktop/Project/clarissa_project/airflow/config/anomalies_detected.json"
sale_data_filtered = sale_data_filtered[["Seller SKU", "Product Name"]]
sale_data_filtered.to_json(json_file_path, orient = 'records', lines=True, force_ascii=False)

In [68]:
# FIlter each SKU I found lexical error in the table
filtered_anomaly = sale_data_filtered[sale_data_filtered['Seller SKU'].str.match("ND+[0-9]", na=False)]
filtered_anomaly[['Seller SKU', 'Product Name', 'Variation']]

,Seller SKU,Product Name,Variation
2921,ND001,Nadia shirt-เสื้อเชิ้ตครอปแขนเบิ้ลแต่งกระเป๋า,Black
2979,ND002,Nadia shirt-เสื้อเชิ้ตครอปแขนเบิ้ลแต่งกระเป๋า,Cream


In [77]:
# Specify each SKU and replace with current one, we can export this dict into json file and call to use
mapping = {
    'ACF001': 'ACF01', 'ACF002': 'ACF02', 'ACF003': 'ACF03', 'MA001': 'AAX01',
    'MA002': 'AAX02', 'MA003': 'AAX03', 'MA005': 'AAX05', 'MA006': 'AAX06',
    'MA007': 'AAX07', 'CY001': 'AAH01', 'CY002': 'AAH02', 'CY003': 'AAH03',
    'CY004': 'AAH04', 'MG001': 'ABA01', 'MG002': 'ABA02', 'MG003': 'ABA03',
    'MG004': 'ABA04', 'MG005': 'ABA05', 'MG006': 'ABA06', 'TW001': 'ABE01',
    'TW002': 'ABE02', 'TW003': 'ABE03', 'TW004': 'ABE05', 'TW005': 'ABE06',
    'TW007': 'ABE04', 'ABN': 'ABN01', 'CH007': 'AAF14', 'CH0012': 'AAF22',
    'CH0011': 'AAF21', 'CH003': 'AAF07', 'CH002': 'AAF03', 'CH004': 'AAF10',
    'CH001': 'AAF02', 'CA001': 'AAE01', 'CA002': 'AAE02', 'AS001': 'AAC01',
    'AC002': 'ABJ02', 'AC002': 'ABJ01', 'LU007': 'AAW09', 'LU018': 'AAW19',
    'LU016': 'AAW17', 'LU010': 'AAW14', 'LU008': 'AAW10', 'LU017': 'AAW18',
    'LU014': 'AAW04', 'LU003': 'AAW03', 'LU019': 'AAW20', 'LU002': 'AAW02',
    'LU013': 'AAW12', 'LU009': 'AAW11', 'LU011': 'AAW15', 'LU001': 'AAW01',
    'LU015': 'AAW13', 'LU012': 'AAW16', 'LU004': 'AAW05', 'LU006': 'AAW07',
    'LU005': 'AAW06', 'IR002': 'AAL02', 'IR003': 'AAL03', 'IR001': 'AAL01',
    'IR004': 'AAL04', 'IR005': 'AAL05', 'JR001': 'AAO01', 'JR002': 'AAO02',
    'AL005': 'AAA05', 'AL006': 'AAA06', 'AL004': 'AAA04', 'AL003': 'AAA03',
    'AL001': 'AAA01', 'AL002': 'AAA02', 'HS006': 'ABI15', 'HS007': 'ABI14',
    'HS009': 'ABI03', 'HS008': 'ABI02', 'HS001': 'ABI01', 'HS003': 'ABI06',
    'ZP004': 'ABH04', 'ZP003': 'ABH03', 'ZP002': 'ABH02', 'ZP001': 'ABH01',
    'MC001': 'AAZ01', 'MC004': 'AAZ04', 'MC005': 'AAZ05', 'MC002': 'AAZ02',
    'MC003': 'AAZ03', 'LN001': 'AAT01', 'LN003': 'AAT03', 'LN002': 'AAT02',
    'WD004': 'ABF04', 'WD003': 'ABF03', 'WD001': 'ABF01', 'WD002': 'ABF02',
    'LS005': 'AAV06', 'LS008': 'AAV10', 'LS009': 'AAV11', 'LS002': 'AAV02',
    'LS006': 'AAV07', 'JN004': 'AAN04', 'JN003': 'AAN03', 'JN002': 'AAN02',
    'JN001': 'AAN01', 'JN005': 'AAN05', 'HP002': 'AAK03', 'HP008': 'AAK06',
    'HP005': 'AAK08', 'HP003': 'AAK07', 'HP004': 'AAK10', 'HP001': 'AAK02',
    'HP007': 'AAK04', 'HP006': 'AAK12', 'JS001': 'AAP01', 'JS002': 'AAP02',
    'SS001': 'ABD01', 'SS003': 'ABD03', 'SS002': 'ABD02', 'AP001': 'AAB01',
    'AP002': 'AAB02', 'AP003': 'AAB03', 'AP004': 'AAB04', 'MB003': 'AAY03',
    'MB001': 'AAY01', 'MB004': 'AAY04', 'MB002': 'AAY02', 'AC001': 'ABJ01',
    'EM001': 'AAI01', 'EM002': 'AAI02', 'LL003': 'AAS03', 'LL002': 'AAS02',
    'LL001': 'AAS01', 'CT003': 'AAG03', 'FF001': 'AAJ01', 'FF002': 'AAJ02',
    'LR001': 'ABK01', 'BD002': 'AAD02', 'BD003': 'AAD03', 'BD001': 'AAD01',
    'JU005': 'AAQ05', 'JU004': 'AAQ04', 'JU002': 'AAQ02', 'JU003': 'AAQ03',
    'JU001': 'AAQ01', 'ZO001': 'ABG01', 'ZO002': 'ABG02', 'ZO003': 'ABG03',
    'KI004': 'AAR04', 'KI003': 'AAR03', 'KI001': 'AAR01', 'KI002': 'AAR02',
    'ND001': 'ABB01', 'ND002': 'ABB02', 'AAH05': 'AAH09', 'AAH06': 'AAH08'
}

sku_file_path = "/Users/fiat/Desktop/Project/Clarissa_project/airflow/config/sku.json"

with open(sku_file_path, 'w') as json_file:
    json.dump(mapping, json_file, indent=4)

In [70]:
# load json and use as dict to change all old SKU to latest one
with open(sku_file_path, 'r') as json_file:
    sku_mapping = json.load(json_file)

# Replace Thai letter on filtered
sale_data_filtered['Seller SKU'] = sale_data_filtered['Seller SKU'].str.replace(r'[\u0E00-\u0E7F]+', '', regex=True)

# Replace the values in the 'Seller SKU' column using the mapping dictionary
sale_data_filtered['Seller SKU'] = sale_data_filtered['Seller SKU'].replace(sku_mapping)

# After dropping mapping, filter only anamoly values
sale_data_filtered = sale_data_filtered[~sale_data_filtered['Seller SKU'].str.match(pattern, na=False)] # pattern = r'^[a-zA-Z]{3}[0-9]{2}$' from above cell
sale_data_filtered[['Seller SKU', 'Product Name', 'Variation']]

,Seller SKU,Product Name,Variation


In [72]:
# Check result
sale_data_new_df.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Cancelled Time,Cancel By,Tracking ID,Buyer Username,Country,Province,District,Payment Method,Product Category
0,577616838816336780,Shipped,In Transit,NaN,ABX08,Hannah shirt - เสื้อคอปกผ้าร่องพรีเมียม,White,1,0,190.00,190.00,0.00,0.00,190.00,0.00,35.00,0.00,35.00,0.00,NaN,190.00,NaN,22/07/2023 17:49:40\t,\t,NaN,7.271803e+11,pavena607,ไทย,ตาก,บ้านตาก,Mbanking,Blouses & Shirts
1,577616009599683475,Shipped,In Transit,NaN,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,0,260.00,260.00,0.00,0.00,260.00,0.00,35.00,0.00,35.00,0.00,NaN,260.00,NaN,22/07/2023 14:39:04\t,\t,NaN,7.271720e+11,warangkanasmile,ไทย,เชียงใหม่,เมืองเชียงใหม่,Cash on delivery,Jackets & Coats
2,577604062025123857,Canceled,Canceled,Cancel,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,1,260.00,260.00,0.00,120.00,140.00,0.00,35.00,0.00,35.00,0.00,NaN,140.00,140.00,20/07/2023 01:51:03\t,20/07/2023 02:55:11\t,User,7.271079e+11,yp091520,ไทย,ภูเก็ต,เมืองภูเก็ต,Cash on delivery,Jackets & Coats
3,577600144372763216,Shipped,Delivered,NaN,ACS02,Cristine shirt - เสื้อเชิ้ตแขนยาวผูกคอ,Ocean blue,1,0,190.00,190.00,0.00,70.00,120.00,0.00,35.00,0.00,35.00,0.00,NaN,120.00,NaN,19/07/2023 10:22:09\t,\t,NaN,7.271745e+11,nasaneen28,ราชอาณาจักรไทย,ปัตตานี,เมืองปัตตานี,Cash on delivery,Blouses & Shirts
4,577594882275051750,Shipped,Delivered,NaN,AAK05,Harmony pants - กางเกงขายาวเดินตะเข็บ,"White, S",1,0,290.00,290.00,0.00,0.00,290.00,0.00,35.00,0.00,35.00,0.00,NaN,290.00,NaN,18/07/2023 07:57:52\t,\t,NaN,7.271715e+11,waris1110,ไทย,มหาสารคาม,กันทรวิชัย,Truemoney,Trousers


In [54]:
# Apply to sale_data_new_df
sale_data_new_df['Seller SKU'] = sale_data_new_df['Seller SKU'].replace(mapping)

# Remove Thai letters in Seller SKU column
sale_data_new_df['Seller SKU'] = sale_data_new_df['Seller SKU'].str.replace(r'[\u0E00-\u0E7F]+', '', regex=True)

# Replace \t with ''
sale_data_new_df[['Created Time', 'Cancelled Time']] = sale_data_new_df[['Created Time', 'Cancelled Time']].replace('\t','', regex=True)

In [160]:
# Change 'Created Time' and 'Cancelled Time' to date time format
sale_data_new_df[['Created Time', 'Cancelled Time']] = sale_data_new_df[['Created Time', 'Cancelled Time']].apply(pd.to_datetime, format='%d/%m/%Y %H:%M:%S')

For these cols below change their types to "float" (All finances should be in float):
'SKU Unit Original Price',
'SKU Subtotal Before Discount',
'SKU Platform Discount',
'SKU Seller Discount',
'SKU Subtotal After Discount',
'Shipping Fee After Discount',
'Original Shipping Fee',
'Shipping Fee Seller Discount',
'Shipping Fee Platform Discount',
'Taxes',
'Small Order Fee',
'Order Amount',
'Order Refund Amount',

For these cols below change to "string" (All id should be in string):
'Order ID',
'Tracking ID'

In [161]:
# Use dictionary to change data types
dtype_map = {
    'Order ID': str,
    'Tracking ID': str
}

cols_numeric = [
    'SKU Unit Original Price',
    'SKU Subtotal Before Discount',
    'SKU Platform Discount',
    'SKU Seller Discount',
    'SKU Subtotal After Discount',
    'Shipping Fee After Discount',
    'Original Shipping Fee',
    'Shipping Fee Seller Discount',
    'Shipping Fee Platform Discount',
    'Taxes',
    'Small Order Fee',
    'Order Amount',
    'Order Refund Amount'
]

sale_data_new_df = sale_data_new_df.astype(dtype_map)

# Remove \* from Tracking ID cols
sale_data_new_df['Tracking ID'] = sale_data_new_df['Tracking ID'].replace(r'\..*', '', regex=True)

# Use loop to change all type automatic numeric from cols_numeric
for cols in cols_numeric:
    sale_data_new_df[cols] = pd.to_numeric(sale_data_new_df[cols], errors='coerce')
    
sale_data_new_df.dtypes

Order ID                                  object
Order Status                              object
Order Substatus                           object
Cancelation/Return Type                   object
Seller SKU                                object
Product Name                              object
Variation                                 object
Quantity                                   int64
Sku Quantity of return                     int64
SKU Unit Original Price                  float64
SKU Subtotal Before Discount             float64
SKU Platform Discount                    float64
SKU Seller Discount                      float64
SKU Subtotal After Discount              float64
Shipping Fee After Discount              float64
Original Shipping Fee                    float64
Shipping Fee Seller Discount             float64
Shipping Fee Platform Discount           float64
Taxes                                    float64
Small Order Fee                          float64
Order Amount        

In [162]:
# See all unique value in column country >> Every value is Thailand so we will change all this to only 'Thailand'
country_filtered_df = sale_data_new_df['Country'].drop_duplicates()
country_filtered_df.head()

0                ไทย
3     ราชอาณาจักรไทย
10          Thailand
38            ထိုင်း
Name: Country, dtype: object

In [163]:
# Check all unique value on 'Province'
# We have to transform all province to 'en' but later
province_filtered_df = sale_data_new_df['Province'].drop_duplicates()
province_filtered_df.head()

0          ตาก
1    เชียงใหม่
2       ภูเก็ต
3      ปัตตานี
4    มหาสารคาม
Name: Province, dtype: object

In [164]:
# Check all unique value on 'District'
# We have to work on this one first because if we know district in 'en' we can transform all province to 'en' by dict from API
district_filtered_df = sale_data_new_df['District'].drop_duplicates()
district_filtered_df.head()

0           บ้านตาก
1    เมืองเชียงใหม่
2       เมืองภูเก็ต
3      เมืองปัตตานี
4        กันทรวิชัย
Name: District, dtype: object

In [166]:
# Get province API
url_1 = 'https://raw.githubusercontent.com/kongvut/thai-province-data/master/api_province.json'
r_1 = requests.get(url_1)
result_th_province = r_1.json()
# Get district API
url_2 = 'https://raw.githubusercontent.com/kongvut/thai-province-data/master/api_amphure.json'
r_2 = requests.get(url_2)
result_th_district = r_2.json()

# Convert to pandas
th_province = pd.DataFrame(result_th_province)
th_district = pd.DataFrame(result_th_district)

# Select only usable columns
th_district = th_district[['name_th', 'name_en', 'province_id']]

# Merge district with province
th_address = th_district.merge(th_province, how='left', left_on='province_id', right_on='id')

# Drop unused columns
columns_to_drop = ['province_id', 'id', 'geography_id', 'created_at', 'updated_at', 'deleted_at']
rename_dict = {'name_th_x': 'district_th',
               'name_en_x': 'district_en',
               'name_th_y': 'province_th',
               'name_en_y': 'province_en'}

th_address = th_address.drop(columns_to_drop, axis=1).rename(columns=rename_dict)
th_address['district_th'] = th_address['district_th'].str.replace('^เขต','', regex=True)

# Zip into dict
district_dict = dict(zip(th_address['district_th'], th_address['district_en']))
province_dict = dict(zip(th_address['district_en'], th_address['province_en']))
th_address.head()


,district_th,district_en,province_th,province_en
0,พระนคร,Khet Phra Nakhon,กรุงเทพมหานคร,Bangkok
1,ดุสิต,Khet Dusit,กรุงเทพมหานคร,Bangkok
2,หนองจอก,Khet Nong Chok,กรุงเทพมหานคร,Bangkok
3,บางรัก,Khet Bang Rak,กรุงเทพมหานคร,Bangkok
4,บางเขน,Khet Bang Khen,กรุงเทพมหานคร,Bangkok


In [206]:
# Apply to sale_data_new_df
unused_th = {'^เขต':'' , 'อำเภอ':''}
sale_data_new_df['Country'] = 'Thailand'
sale_data_new_df['District'] = sale_data_new_df['District'].replace(unused_th, regex=True)
sale_data_new_df['District'] = sale_data_new_df['District'].apply(lambda x: district_dict.get(x, x))
sale_data_new_df['Province'] = sale_data_new_df['District'].apply(lambda x: province_dict.get(x, x))
sale_data_new_df.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Cancelled Time,Cancel By,Tracking ID,Buyer Username,Country,Province,District,Payment Method,Product Category
0,577616838816336780,Shipped,In Transit,NaN,ABX08,Hannah shirt - เสื้อคอปกผ้าร่องพรีเมียม,White,1,0,190.0,190.0,0.0,0.0,190.0,0.0,35.0,0.0,35.0,0.0,NaN,190.0,NaN,2023-07-22 17:49:40,NaT,NaN,727180345813,pavena607,Thailand,Tak,Ban Tak,Mbanking,Blouses & Shirts
1,577616009599683475,Shipped,In Transit,NaN,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,0,260.0,260.0,0.0,0.0,260.0,0.0,35.0,0.0,35.0,0.0,NaN,260.0,NaN,2023-07-22 14:39:04,NaT,NaN,727172039856,warangkanasmile,Thailand,Chiang Mai,Mueang Chiang Mai,Cash on delivery,Jackets & Coats
2,577604062025123857,Canceled,Canceled,Cancel,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,1,260.0,260.0,0.0,120.0,140.0,0.0,35.0,0.0,35.0,0.0,NaN,140.0,140.0,2023-07-20 01:51:03,2023-07-20 02:55:11,User,727107904666,yp091520,Thailand,Phuket,Mueang Phuket,Cash on delivery,Jackets & Coats
3,577600144372763216,Shipped,Delivered,NaN,ACS02,Cristine shirt - เสื้อเชิ้ตแขนยาวผูกคอ,Ocean blue,1,0,190.0,190.0,0.0,70.0,120.0,0.0,35.0,0.0,35.0,0.0,NaN,120.0,NaN,2023-07-19 10:22:09,NaT,NaN,727174539582,nasaneen28,Thailand,Pattani,Mueang Pattani,Cash on delivery,Blouses & Shirts
4,577594882275051750,Shipped,Delivered,NaN,AAK05,Harmony pants - กางเกงขายาวเดินตะเข็บ,"White, S",1,0,290.0,290.0,0.0,0.0,290.0,0.0,35.0,0.0,35.0,0.0,NaN,290.0,NaN,2023-07-18 07:57:52,NaT,NaN,727171537481,waris1110,Thailand,Maha Sarakham,Kantharawichai,Truemoney,Trousers


# Revenue data transformation

In [207]:
revenue_data_df.head()

,Formula,Order/adjustment ID,Type,Time(Timezone=UTC),Currency,Subtotal after seller discounts,Subtotal before discounts,Seller discounts,Refund subtotal after seller discounts,Refund subtotal before seller discounts,Refund of seller discounts,Transaction fee,TikTok Shop commission fee,Affiliate commission,Affiliate partner commission,Chargeback,Customer service compensation,Other adjustments,Deductions incurred by Seller,Promotion adjustment,Satisfaction reimbursement,Settlement amount,Related order ID
0,576550368017483850,576550368017483850,Order,29-Jun,THB,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,250.0,/
1,576550377907587120,576550377907587120,Order,29-Jun,THB,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,250.0,/
2,576550391184985073,576550391184985073,Order,29-Jun,THB,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,250.0,/
3,576550498738473619,576550498738473619,Order,29-Jun,THB,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,320.0,/
4,576550492255783632,576550492255783632,Order,29-Jun,THB,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,320.0,/


In [169]:
# Drop unused columns
revenue_data_new_df = revenue_data_df.drop(columns=[
    'Formula',
    'Type',
    'Time(Timezone=UTC)',
    'Currency',
    'Subtotal after seller discounts',
    'Subtotal before discounts',
    'Seller discounts',
    'Refund subtotal after seller discounts',
    'Refund subtotal before seller discounts',
    'Refund of seller discounts',
    'Settlement amount',
    'Related order ID'
    ])

In [170]:
revenue_data_new_df.head()

,Order/adjustment ID,Transaction fee,TikTok Shop commission fee,Affiliate commission,Affiliate partner commission,Chargeback,Customer service compensation,Other adjustments,Deductions incurred by Seller,Promotion adjustment,Satisfaction reimbursement
0,576550368017483850,0.0,0.0,0.0,0,0,0,0,0,0,0
1,576550377907587120,0.0,0.0,0.0,0,0,0,0,0,0,0
2,576550391184985073,0.0,0.0,0.0,0,0,0,0,0,0,0
3,576550498738473619,0.0,0.0,0.0,0,0,0,0,0,0,0
4,576550492255783632,0.0,0.0,0.0,0,0,0,0,0,0,0


In [172]:
# Sum all row that contain nulls
revenue_data_df_filtered = revenue_data_new_df.isna().sum()
revenue_data_df_filtered

Order/adjustment ID              0
Transaction fee                  0
TikTok Shop commission fee       0
Affiliate commission             0
Affiliate partner commission     0
Chargeback                       0
Customer service compensation    0
Other adjustments                0
Deductions incurred by Seller    0
Promotion adjustment             0
Satisfaction reimbursement       0
dtype: int64

In [173]:
# Check column of revenue_data_df return as list
revenue_data_new_df.columns

Index(['Order/adjustment ID', 'Transaction fee', 'TikTok Shop commission fee',
       'Affiliate commission', 'Affiliate partner commission', 'Chargeback',
       'Customer service compensation', 'Other adjustments',
       'Deductions incurred by Seller', 'Promotion adjustment',
       'Satisfaction reimbursement'],
      dtype='object')

In [209]:
# Change all type to float and convert to positive except Order ID, Order ID to string
for column in revenue_data_new_df.columns:
    if column != 'Order/adjustment ID':
        revenue_data_new_df[column] = revenue_data_new_df[column].astype(float).abs()
        
revenue_data_new_df['Order/adjustment ID'] = revenue_data_new_df['Order/adjustment ID'].astype(str)

revenue_data_new_df


,Order/adjustment ID,Transaction fee,TikTok Shop commission fee,Affiliate commission,Affiliate partner commission,Chargeback,Customer service compensation,Other adjustments,Deductions incurred by Seller,Promotion adjustment,Satisfaction reimbursement
0,576550368017483850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,576550377907587120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,576550391184985073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,576550498738473619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,576550492255783632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2165,577386513602808815,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2166,577369449466726955,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2167,577373411475294428,12.0,10.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2168,577360446915512337,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [175]:
# Drop null value on column 'Order/adjustment ID'
revenue_data_new_df = revenue_data_new_df.dropna(subset=['Order/adjustment ID'])

In [176]:
# Drop duplicate Order ID because in this dataframe Order ID must not duplicate
revenue_data_new_df['Order/adjustment ID'] = revenue_data_new_df['Order/adjustment ID'].drop_duplicates()

In [177]:
# Check type of each column
revenue_data_new_df.dtypes

Order/adjustment ID               object
Transaction fee                  float64
TikTok Shop commission fee       float64
Affiliate commission             float64
Affiliate partner commission     float64
Chargeback                       float64
Customer service compensation    float64
Other adjustments                float64
Deductions incurred by Seller    float64
Promotion adjustment             float64
Satisfaction reimbursement       float64
dtype: object

# Inbound data transformation

In [178]:
# Sperate read from another 2 csv because this Dataframe has to skip first row (It is not using)
blob = bucket.blob(f"{blob_folder_path}/{inbound_data}")
csv_file = blob.download_as_text()
inbound_data_df = pd.read_csv(io.StringIO(csv_file), skiprows=1)

In [208]:
inbound_data_df

,Order,Date,Unnamed: 2,Product name,Color,Size,Color + Size,QTY,Cost,Total,SKU,Location,SKU + Location,Product - Color - Size,Shop name,Batch,Batch+SKU
0,1.0,18/06/2022,Jun,Mable Shirt - เสื้อเชิ้ตผ้าย่นคอตตอน,Mint,Freesize,Mint - Freesize,2.0,150.0,300,AAX01,A2,AAX01-A2,Mable Shirt - เสื้อเชิ้ตผ้าย่นคอตตอน - Mint - ...,Seppy,CL220618,CL220618-AAX01
1,2.0,18/06/2022,Jun,Mable Shirt - เสื้อเชิ้ตผ้าย่นคอตตอน,Oak,Freesize,Oak - Freesize,2.0,150.0,300,AAX06,A2,AAX06-A2,Mable Shirt - เสื้อเชิ้ตผ้าย่นคอตตอน - Oak - F...,Seppy,CL220618,CL220618-AAX06
2,3.0,18/06/2022,Jun,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า,Yellow,Freesize,Yellow - Freesize,2.0,150.0,300,AAA01,A4,AAA01-A4,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า - Yell...,Seppy,CL220618,CL220618-AAA01
3,4.0,18/06/2022,Jun,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า,Black,Freesize,Black - Freesize,3.0,150.0,450,AAA02,A4,AAA02-A4,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า - Blac...,Seppy,CL220618,CL220618-AAA02
4,5.0,18/06/2022,Jun,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า,Cream,Freesize,Cream - Freesize,3.0,150.0,450,AAA03,A4,AAA03-A4,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า - Crea...,Seppy,CL220618,CL220618-AAA03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,0,NaN,NaN,-,- -,NaN,NaN,NaN
1993,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,0,NaN,NaN,-,- -,NaN,NaN,NaN
1994,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,0,NaN,NaN,-,- -,NaN,NaN,NaN
1995,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,0,NaN,NaN,-,- -,NaN,NaN,NaN


In [180]:
# Drop unsused columns (In this case I have to find NaN column with float('nan') to delete 'nan' column)
# Drop NaN rows subset = SKU
columns_to_drop = [
    'Order',
    'Unnamed: 2',
    'Color + Size',
    'Location',
    'SKU + Location',
    'Product - Color - Size',
    'Batch+SKU'
]
inbound_data_new_df = inbound_data_df.drop(columns=columns_to_drop).dropna(subset='SKU')

In [181]:
# Check type of all data
inbound_data_new_df.dtypes

Date             object
Product name     object
Color            object
Size             object
QTY             float64
Cost            float64
Total             int64
SKU              object
Shop name        object
Batch            object
dtype: object

In [182]:
# Change type of column 'Date' to date
inbound_data_new_df['Date'] = pd.to_datetime(inbound_data_new_df['Date'], format='%d/%m/%Y').dt.date
inbound_data_new_df['QTY'] = inbound_data_new_df['QTY'].astype(int)
cols_numeric = ['Cost', 'Total']
for col in cols_numeric:
    inbound_data_new_df[col] = inbound_data_new_df[col].astype(float)
    
inbound_data_new_df.dtypes

Date             object
Product name     object
Color            object
Size             object
QTY               int64
Cost            float64
Total           float64
SKU              object
Shop name        object
Batch            object
dtype: object

In [214]:
# find missing values on all columns
missing_values = inbound_data_new_df.isna().sum()
print(missing_values)

Date            0
Product name    0
Color           0
Size            0
QTY             0
Cost            0
Total           0
SKU             0
Shop name       0
Batch           2
dtype: int64


In [185]:
# Check if column 'Date' is date_value or not, using type() (Because in pandas shows datetime.date as Object)
date_value = inbound_data_new_df['Date'].iloc[0]
print(type(date_value))

<class 'datetime.date'>


In [210]:
inbound_data_new_df

,Date,Product name,Color,Size,QTY,Cost,Total,SKU,Shop name,Batch
0,2022-06-18,Mable Shirt - เสื้อเชิ้ตผ้าย่นคอตตอน,Mint,Freesize,2,150.0,300.0,AAX01,Seppy,CL220618
1,2022-06-18,Mable Shirt - เสื้อเชิ้ตผ้าย่นคอตตอน,Oak,Freesize,2,150.0,300.0,AAX06,Seppy,CL220618
2,2022-06-18,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า,Yellow,Freesize,2,150.0,300.0,AAA01,Seppy,CL220618
3,2022-06-18,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า,Black,Freesize,3,150.0,450.0,AAA02,Seppy,CL220618
4,2022-06-18,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า,Cream,Freesize,3,150.0,450.0,AAA03,Seppy,CL220618
...,...,...,...,...,...,...,...,...,...,...
611,2023-01-27,Bestie shirt - เสื้อเชิ้ตครอปผ้านิ่ม,Milk tea,Freesize,10,150.0,1500.0,ACZ05,Seppy,CL230127
612,2023-01-27,Peony shirt - เสื้อเชิ้ตผ้านิ่มลายดอก,Light pink,Freesize,6,130.0,780.0,ADA01,Seppy,CL230127
613,2023-01-27,Sweetie shirt - เสื้อคอเหลี่ยมลายดอก,Cream,Freesize,10,130.0,1300.0,ADB01,Seppy,CL230127
614,2023-01-27,Ribbon shirt - เสื้อเชิ้ตแขนรูด,Charcoal,Freesize,4,180.0,720.0,ACY01,Vora,CL230127


# Merge all data

In [186]:
sale_data_new_df.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Cancelled Time,Cancel By,Tracking ID,Buyer Username,Country,Province,District,Payment Method,Product Category
0,577616838816336780,Shipped,In Transit,NaN,ABX08,Hannah shirt - เสื้อคอปกผ้าร่องพรีเมียม,White,1,0,190.0,190.0,0.0,0.0,190.0,0.0,35.0,0.0,35.0,0.0,NaN,190.0,NaN,2023-07-22 17:49:40,NaT,NaN,727180345813,pavena607,Thailand,Tak,Ban Tak,Mbanking,Blouses & Shirts
1,577616009599683475,Shipped,In Transit,NaN,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,0,260.0,260.0,0.0,0.0,260.0,0.0,35.0,0.0,35.0,0.0,NaN,260.0,NaN,2023-07-22 14:39:04,NaT,NaN,727172039856,warangkanasmile,Thailand,Chiang Mai,Mueang Chiang Mai,Cash on delivery,Jackets & Coats
2,577604062025123857,Canceled,Canceled,Cancel,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,1,260.0,260.0,0.0,120.0,140.0,0.0,35.0,0.0,35.0,0.0,NaN,140.0,140.0,2023-07-20 01:51:03,2023-07-20 02:55:11,User,727107904666,yp091520,Thailand,Phuket,Mueang Phuket,Cash on delivery,Jackets & Coats
3,577600144372763216,Shipped,Delivered,NaN,ACS02,Cristine shirt - เสื้อเชิ้ตแขนยาวผูกคอ,Ocean blue,1,0,190.0,190.0,0.0,70.0,120.0,0.0,35.0,0.0,35.0,0.0,NaN,120.0,NaN,2023-07-19 10:22:09,NaT,NaN,727174539582,nasaneen28,Thailand,Pattani,Mueang Pattani,Cash on delivery,Blouses & Shirts
4,577594882275051750,Shipped,Delivered,NaN,AAK05,Harmony pants - กางเกงขายาวเดินตะเข็บ,"White, S",1,0,290.0,290.0,0.0,0.0,290.0,0.0,35.0,0.0,35.0,0.0,NaN,290.0,NaN,2023-07-18 07:57:52,NaT,NaN,727171537481,waris1110,Thailand,Maha Sarakham,Kantharawichai,Truemoney,Trousers


In [187]:
# Drop duplicate first before merge
inbound_data_to_merge = inbound_data_new_df.drop_duplicates(subset='SKU')
inbound_data_to_merge.head()

,Date,Product name,Color,Size,QTY,Cost,Total,SKU,Shop name,Batch
0,2022-06-18,Mable Shirt - เสื้อเชิ้ตผ้าย่นคอตตอน,Mint,Freesize,2,150.0,300.0,AAX01,Seppy,CL220618
1,2022-06-18,Mable Shirt - เสื้อเชิ้ตผ้าย่นคอตตอน,Oak,Freesize,2,150.0,300.0,AAX06,Seppy,CL220618
2,2022-06-18,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า,Yellow,Freesize,2,150.0,300.0,AAA01,Seppy,CL220618
3,2022-06-18,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า,Black,Freesize,3,150.0,450.0,AAA02,Seppy,CL220618
4,2022-06-18,Alley Shirt - เสื้อเชิ้ตครอปกระเป๋าหน้า,Cream,Freesize,3,150.0,450.0,AAA03,Seppy,CL220618


In [188]:
revenue_data_new_df.head()

,Order/adjustment ID,Transaction fee,TikTok Shop commission fee,Affiliate commission,Affiliate partner commission,Chargeback,Customer service compensation,Other adjustments,Deductions incurred by Seller,Promotion adjustment,Satisfaction reimbursement
0,576550368017483850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,576550377907587120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,576550391184985073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,576550498738473619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,576550492255783632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


I will merge all sale data with just SKU, Cost, Shop name from Inbound data
The whole Inbound data would show in different Visualization

In [189]:
# Merge sale and inbound then drop column 'SKU'
sale_inbound_merged = sale_data_new_df.merge(inbound_data_to_merge[['SKU', 'Cost', 'Shop name']], how='left', left_on='Seller SKU', right_on='SKU').drop(columns=['SKU'])
sale_inbound_merged.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Cancelled Time,Cancel By,Tracking ID,Buyer Username,Country,Province,District,Payment Method,Product Category,Cost,Shop name
0,577616838816336780,Shipped,In Transit,NaN,ABX08,Hannah shirt - เสื้อคอปกผ้าร่องพรีเมียม,White,1,0,190.0,190.0,0.0,0.0,190.0,0.0,35.0,0.0,35.0,0.0,NaN,190.0,NaN,2023-07-22 17:49:40,NaT,NaN,727180345813,pavena607,Thailand,Tak,Ban Tak,Mbanking,Blouses & Shirts,115.0,Pech Production
1,577616009599683475,Shipped,In Transit,NaN,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,0,260.0,260.0,0.0,0.0,260.0,0.0,35.0,0.0,35.0,0.0,NaN,260.0,NaN,2023-07-22 14:39:04,NaT,NaN,727172039856,warangkanasmile,Thailand,Chiang Mai,Mueang Chiang Mai,Cash on delivery,Jackets & Coats,220.0,Maria
2,577604062025123857,Canceled,Canceled,Cancel,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,1,260.0,260.0,0.0,120.0,140.0,0.0,35.0,0.0,35.0,0.0,NaN,140.0,140.0,2023-07-20 01:51:03,2023-07-20 02:55:11,User,727107904666,yp091520,Thailand,Phuket,Mueang Phuket,Cash on delivery,Jackets & Coats,220.0,Maria
3,577600144372763216,Shipped,Delivered,NaN,ACS02,Cristine shirt - เสื้อเชิ้ตแขนยาวผูกคอ,Ocean blue,1,0,190.0,190.0,0.0,70.0,120.0,0.0,35.0,0.0,35.0,0.0,NaN,120.0,NaN,2023-07-19 10:22:09,NaT,NaN,727174539582,nasaneen28,Thailand,Pattani,Mueang Pattani,Cash on delivery,Blouses & Shirts,170.0,T2Y
4,577594882275051750,Shipped,Delivered,NaN,AAK05,Harmony pants - กางเกงขายาวเดินตะเข็บ,"White, S",1,0,290.0,290.0,0.0,0.0,290.0,0.0,35.0,0.0,35.0,0.0,NaN,290.0,NaN,2023-07-18 07:57:52,NaT,NaN,727171537481,waris1110,Thailand,Maha Sarakham,Kantharawichai,Truemoney,Trousers,200.0,Cosmo


In [190]:
# Merge revenue_data with merged sale_inbound_merged dataframe then drop 'Order/adjustment ID'
merged_all_data = sale_inbound_merged.merge(revenue_data_new_df, how='left', left_on='Order ID', right_on='Order/adjustment ID').drop(columns=['Order/adjustment ID'])
merged_all_data.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Cancelled Time,Cancel By,Tracking ID,Buyer Username,Country,Province,District,Payment Method,Product Category,Cost,Shop name,Transaction fee,TikTok Shop commission fee,Affiliate commission,Affiliate partner commission,Chargeback,Customer service compensation,Other adjustments,Deductions incurred by Seller,Promotion adjustment,Satisfaction reimbursement
0,577616838816336780,Shipped,In Transit,NaN,ABX08,Hannah shirt - เสื้อคอปกผ้าร่องพรีเมียม,White,1,0,190.0,190.0,0.0,0.0,190.0,0.0,35.0,0.0,35.0,0.0,NaN,190.0,NaN,2023-07-22 17:49:40,NaT,NaN,727180345813,pavena607,Thailand,Tak,Ban Tak,Mbanking,Blouses & Shirts,115.0,Pech Production,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,577616009599683475,Shipped,In Transit,NaN,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,0,260.0,260.0,0.0,0.0,260.0,0.0,35.0,0.0,35.0,0.0,NaN,260.0,NaN,2023-07-22 14:39:04,NaT,NaN,727172039856,warangkanasmile,Thailand,Chiang Mai,Mueang Chiang Mai,Cash on delivery,Jackets & Coats,220.0,Maria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,577604062025123857,Canceled,Canceled,Cancel,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,1,260.0,260.0,0.0,120.0,140.0,0.0,35.0,0.0,35.0,0.0,NaN,140.0,140.0,2023-07-20 01:51:03,2023-07-20 02:55:11,User,727107904666,yp091520,Thailand,Phuket,Mueang Phuket,Cash on delivery,Jackets & Coats,220.0,Maria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,577600144372763216,Shipped,Delivered,NaN,ACS02,Cristine shirt - เสื้อเชิ้ตแขนยาวผูกคอ,Ocean blue,1,0,190.0,190.0,0.0,70.0,120.0,0.0,35.0,0.0,35.0,0.0,NaN,120.0,NaN,2023-07-19 10:22:09,NaT,NaN,727174539582,nasaneen28,Thailand,Pattani,Mueang Pattani,Cash on delivery,Blouses & Shirts,170.0,T2Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,577594882275051750,Shipped,Delivered,NaN,AAK05,Harmony pants - กางเกงขายาวเดินตะเข็บ,"White, S",1,0,290.0,290.0,0.0,0.0,290.0,0.0,35.0,0.0,35.0,0.0,NaN,290.0,NaN,2023-07-18 07:57:52,NaT,NaN,727171537481,waris1110,Thailand,Maha Sarakham,Kantharawichai,Truemoney,Trousers,200.0,Cosmo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
# I have to this sale because in the first place the output rows were increased, fixed by drop_duplicated for revenue_data before output to GCS
increased_rows = revenue_data_new_df[~revenue_data_new_df['Order/adjustment ID'].isin(merged_all_data['Order ID'])]
increased_rows.head()

,Order/adjustment ID,Transaction fee,TikTok Shop commission fee,Affiliate commission,Affiliate partner commission,Chargeback,Customer service compensation,Other adjustments,Deductions incurred by Seller,Promotion adjustment,Satisfaction reimbursement
9,576550377500347040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,576552425684306113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,576552408491526359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56,576555335741179949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,576555283814058227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [193]:
# Put all coloumn to variable
revenue_columns = revenue_data_new_df.columns
revenue_columns

Index(['Order/adjustment ID', 'Transaction fee', 'TikTok Shop commission fee',
       'Affiliate commission', 'Affiliate partner commission', 'Chargeback',
       'Customer service compensation', 'Other adjustments',
       'Deductions incurred by Seller', 'Promotion adjustment',
       'Satisfaction reimbursement'],
      dtype='object')

In [194]:
# From revenue_data.columns, loop to get all column name except Order ID then use .mask to make condition which 'Order ID' is duplicated, and replace 'cols' = np.nan in True value
for cols in revenue_columns:
    if cols != 'Order/adjustment ID':
        merged_all_data[cols] = merged_all_data[cols].mask(merged_all_data['Order ID'].duplicated(), 0)

In [195]:
# I have to this sale because in the first place the output rows were increased, fixed by drop_duplicated for revenue_data before output to GCS
increased_rows = revenue_data_new_df[~revenue_data_new_df['Order/adjustment ID'].isin(merged_all_data['Order ID'])]
increased_rows

,Order/adjustment ID,Transaction fee,TikTok Shop commission fee,Affiliate commission,Affiliate partner commission,Chargeback,Customer service compensation,Other adjustments,Deductions incurred by Seller,Promotion adjustment,Satisfaction reimbursement
9,576550377500347040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,576552425684306113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,576552408491526359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56,576555335741179949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,576555283814058227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,576557124174449366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,576555601935501643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188,576560457300216516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193,576560407903701476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
210,576557788239726994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [201]:
# Check null values on payment method
merged_all_data_filtered = merged_all_data['Payment Method'].isnull().sum()
merged_all_data_filtered

252

In [197]:
# Check last output
merged_all_data.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Seller SKU,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,SKU Platform Discount,SKU Seller Discount,SKU Subtotal After Discount,Shipping Fee After Discount,Original Shipping Fee,Shipping Fee Seller Discount,Shipping Fee Platform Discount,Taxes,Small Order Fee,Order Amount,Order Refund Amount,Created Time,Cancelled Time,Cancel By,Tracking ID,Buyer Username,Country,Province,District,Payment Method,Product Category,Cost,Shop name,Transaction fee,TikTok Shop commission fee,Affiliate commission,Affiliate partner commission,Chargeback,Customer service compensation,Other adjustments,Deductions incurred by Seller,Promotion adjustment,Satisfaction reimbursement
0,577616838816336780,Shipped,In Transit,NaN,ABX08,Hannah shirt - เสื้อคอปกผ้าร่องพรีเมียม,White,1,0,190.0,190.0,0.0,0.0,190.0,0.0,35.0,0.0,35.0,0.0,NaN,190.0,NaN,2023-07-22 17:49:40,NaT,NaN,727180345813,pavena607,Thailand,Tak,Ban Tak,Mbanking,Blouses & Shirts,115.0,Pech Production,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,577616009599683475,Shipped,In Transit,NaN,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,0,260.0,260.0,0.0,0.0,260.0,0.0,35.0,0.0,35.0,0.0,NaN,260.0,NaN,2023-07-22 14:39:04,NaT,NaN,727172039856,warangkanasmile,Thailand,Chiang Mai,Mueang Chiang Mai,Cash on delivery,Jackets & Coats,220.0,Maria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,577604062025123857,Canceled,Canceled,Cancel,ACR01,Chertam jacket - เสื้อแจ็คเก็ตครอปเอวสม็อค,Black,1,1,260.0,260.0,0.0,120.0,140.0,0.0,35.0,0.0,35.0,0.0,NaN,140.0,140.0,2023-07-20 01:51:03,2023-07-20 02:55:11,User,727107904666,yp091520,Thailand,Phuket,Mueang Phuket,Cash on delivery,Jackets & Coats,220.0,Maria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,577600144372763216,Shipped,Delivered,NaN,ACS02,Cristine shirt - เสื้อเชิ้ตแขนยาวผูกคอ,Ocean blue,1,0,190.0,190.0,0.0,70.0,120.0,0.0,35.0,0.0,35.0,0.0,NaN,120.0,NaN,2023-07-19 10:22:09,NaT,NaN,727174539582,nasaneen28,Thailand,Pattani,Mueang Pattani,Cash on delivery,Blouses & Shirts,170.0,T2Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,577594882275051750,Shipped,Delivered,NaN,AAK05,Harmony pants - กางเกงขายาวเดินตะเข็บ,"White, S",1,0,290.0,290.0,0.0,0.0,290.0,0.0,35.0,0.0,35.0,0.0,NaN,290.0,NaN,2023-07-18 07:57:52,NaT,NaN,727171537481,waris1110,Thailand,Maha Sarakham,Kantharawichai,Truemoney,Trousers,200.0,Cosmo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
# Create schema field as json file for easier editing
schema_fields = [
            {
                "mode": "NULLABLE",
                "name": "Order_ID",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Order_Status",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Order_Substatus",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Cancelation_or_Return_Type",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Seller_SKU",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Product_Name",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Variation",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Quantity",
                "type": "INTEGER"
            },
            {
                "mode": "NULLABLE",
                "name": "SKU_Quantity_of_return",
                "type": "INTEGER"
            },
            {
                "mode": "NULLABLE",
                "name": "SKU_Unit_Original_Price",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "SKU_Subtotal_Before_Discount",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "SKU_Platform_Discount",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "SKU_Seller_Discount",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "SKU_Subtotal_After_Discount",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Shipping_Fee_After_Discount",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Original_Shipping_Fee",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Shipping_Fee_Seller_Discount",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Shipping_Fee_Platform_Discount",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Taxes",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Small_Order_Fee",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Order_Amount",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Order_Refund_Amount",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Created_Time",
                "type": "TIMESTAMP"
            },
            {
                "mode": "NULLABLE",
                "name": "Cancelled_Time",
                "type": "TIMESTAMP"
            },
            {
                "mode": "NULLABLE",
                "name": "Cancel_By",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Tracking_ID",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Buyer_Username",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Country",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Province",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "District",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Payment_Method",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Product_Category",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Cost",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Shop_name",
                "type": "STRING"
            },
            {
                "mode": "NULLABLE",
                "name": "Transaction_Fee",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Tiktok_Shop_Commission_Fee",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Affiliate_Commission",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Affiliate_Partner_Commission",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Charge_Back",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Customer_Service_Compensation",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Other_Adjustments",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Deductions_incurred_by_Seller",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Promotion_adjustment",
                "type": "FLOAT"
            },
            {
                "mode": "NULLABLE",
                "name": "Satisfaction_reimbursement",
                "type": "FLOAT"
            },
        ]

with open('/Users/fiat/Desktop/Project/Clarissa_project/airflow/config/schema_fields.json', 'w') as json_file:
    json.dump(schema_fields, json_file, indent=4)